In [1]:
import os
import cv2
from skimage.transform import resize,rescale
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import pickle

from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.3.4)


In [2]:
input_img=Input(shape=(256,256,3))
l1=Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(input_img)
l2=Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l1)
l3=MaxPooling2D(padding='same')(l2)
l4=Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l3)
l5=Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l4)
l6=MaxPooling2D(padding='same')(l5)
l7=Conv2D(256,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l6)

In [3]:
l8=UpSampling2D()(l7)
l9=Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l8)
l10=Conv2D(128,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l9)
l11=add([l5,l10])
l12=UpSampling2D()(l11)
l13=Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l12)
l14=Conv2D(64,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l13)
l15=add([l14,l2])
decoded=Conv2D(3,(3,3),padding='same',activation='relu',activity_regularizer=regularizers.l1(10e-10))(l15)
autoencoder=Model(input_img,decoded)
autoencoder.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [4]:
autoencoder.compile(optimizer='adadelta',loss='mean_squared_error')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
folder_path = '/content/drive/My Drive/dataset/FaceImages/'

In [13]:
images = os.listdir(folder_path)

In [14]:
for i in range(len(images)):
  images[i]=folder_path+images[i]

In [6]:
images = open('images.pickle', 'rb')
images = pickle.load(images)

In [7]:
def pixalate_image(image, scale_percent = 25):
  width = int(image.shape[1] * scale_percent / 100)
  height = int(image.shape[0] * scale_percent / 100)
  dim = (width, height)
  small_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  width = int(small_image.shape[1] * 100 / scale_percent)
  height = int(small_image.shape[0] * 100 / scale_percent)
  dim = (width, height)
  low_res_image = cv2.resize(small_image, dim, interpolation =  cv2.INTER_AREA)
  return low_res_image

In [8]:
autoencoder.load_weights('/content/checkpoint.h5')

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=400, verbose=2, mode='min')
model_checkpoint =  ModelCheckpoint('checkpoint.h5', save_best_only = True)

counter=1
batch_size=256
final_images=[images[0]]

while counter<len(images):
  if counter%batch_size!=0:
    final_images.append(images[counter])
    counter+=1
  else:
    final_images=[resize(pyplot.imread(i),(256,256)) for i in final_images]
    final_images=np.array(final_images)

    train,test=train_test_split(final_images, random_state = 32, test_size=0.2)
    train_lr = []
    for i in range(train.shape[0]):
      temp = pixalate_image(train[i,:,:,:])
      train_lr.append(temp)
    train_lr = np.array(train_lr)

    test_lr = []
    for i in range(test.shape[0]):
      temp = pixalate_image(test[i,:,:,:])
      test_lr.append(temp)
    test_lr = np.array(test_lr)

    history = autoencoder.fit(train_lr,train,epochs=10000,validation_data=(test_lr, test),callbacks=[model_checkpoint])
    
    final_images=[]
    print(counter)
    counter+=1

In [ ]:
results=autoencoder.predict(test_lr)
plt.imshow(test_lr[1])

In [ ]:
plt.imshow(results[1])

In [ ]:
plt.imshow(test[1])